In [59]:
from Bio import SeqIO
import sys
import os
import pandas as pd
import numpy as np
from collections import Counter 
import re
import itertools
import fnmatch

main_dir = "/Users/sigalova/Desktop/Chlamydia_pangenome/"

In [ ]:
def get_prot2og_dict(OG_table, genome_ids):

    prot2og = dict()
    og2prot = dict()

    for line in OG_table:
        [group,proteins] = line.rstrip().split(":")
        proteins = proteins.strip().split(" ")
        proteins = [x.split("|")[1] for x in proteins if x.split("|")[0] in genome_ids]
        og2prot[group] = proteins
        for prot in proteins:
            prot2og[prot] = group
            
    return [prot2og, og2prot]

# 1. List of genomes

In [61]:
path = main_dir + "info/genome_info_combined.csv"
genomes = pd.read_csv(path)
genomes.index = genomes["genome_id"]
genome_ids_withOutgroup = genomes["genome_id"].tolist() 
genome_ids_withoutOutgroup = [x for x in genome_ids_withOutgroup if x != "W001"]
genome_ids_nodraft = genomes[(genomes["n_contigs"]==1) & (genomes["genome_id"]!="W001")]["genome_id"].tolist()    # without outgroup, no draft genomes

# 2. Dict: protein_id to OG

In [101]:
# 2.A. only genus Chlamydia (with draft genomes)  
OGs_withoutOutgroup = main_dir + '/data/ogroups2019/raw/groups_wo_outgroup_with_singletons.txt'

ogs = open(OGs_withoutOutgroup, "r")
res = get_prot2og_dict(ogs, genome_ids_withoutOutgroup)
prot2og_withoutOutgroup = res[0]
og2prot_withoutOutgroup = res[1]

# 2.B. only genus Chlamydia (without draft genomes)  
ogs = open(OGs_withoutOutgroup, "r")
res = get_prot2og_dict(ogs, genome_ids_nodraft)
prot2og_nodraft = res[0]
og2prot_nodraft = res[1]

In [113]:
# 2.C. genus Chlamydia + outgroup (with draft genomes)  
OGs_withOutgroup = main_dir + '/data/ogroups2019/raw/groups_with_outgroup_with_singletons.txt'

ogs = open(OGs_withOutgroup, "r")
res = get_prot2og_dict(ogs, genome_ids_withOutgroup)
prot2og_withOutgroup = res[0]
og2prot_withOutgroup = res[1]

# 3. Proteins 

In [73]:
prot = pd.read_csv(main_dir + '/data/gbk_parsed/filt/Chlamydia_proteins.txt', delimiter = "\t")
prot = prot.rename(index = str, columns = {"OrthMCL_genome_code": "genome_id", "OrthMCL_prot_code": "protein_id"})
prot.index = prot["protein_id"]

In [76]:
prot.columns

Index([u'protein_id', u'genome_id', u'product', u'start', u'end', u'strand',
       u'length', u'frameshift', u'nucl_seq', u'translation', u'GO_list',
       u'EC_list'],
      dtype='object')

# 4. OGroups

In [114]:
# 4A. only genus Chlamydia (with draft genomes)

# column names in OG table (=> to add pfam)
col_list_withoutOutgroup = ["group_id", "group_size", "distribution", "length", "all_hypothetical", "product", "with_frameshift", "prod_distr", "trachomatis", "psittaci", "pneumoniae", "pecorum", "abortus", "caviae", "felis", "muridarum", "avium", "gallinacea", "ibidis", "suis", "poikilothermis", "sanzinia", "serpentis", "corallus", "GOdistr_rast", "GO_rast", "ECdistr_rast", "EC_rast"]
# output data.frame and file
ogroups_withoutOutgroup = pd.DataFrame(columns = col_list_withoutOutgroup)
outf = main_dir + '/data/ogroups2019/annotated/groups_without_outgroup_with_singletons_annotated.txt'

i = 0
for group in og2prot_withoutOutgroup.keys():
    
    # list of proteins in the group
    proteins = og2prot_withoutOutgroup[group]
    
    # !! False if some genomes were excluded after OGs had been created !!
    if len(proteins) > 0: 
        
        # prepare
        prod_list = []
        frame_list = []
        genome_list = []
        GO_rast = []
        EC_rast = []
        trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum, avium, gallinacea, ibidis, suis, poikilothermis, sanzinia , serpentis, corallus   = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        product = ''
        distribution = 0
        
        # general statistics
        med_length = prot[prot.index.isin(proteins)]["length"].median()
        OGsize = len(proteins)

        # info by protein
        for p in proteins:

            #products
            prod = prot.loc[p,"product"]
            prod_list.append(prod)

            #frameshifts
            frame = prot.loc[p, "frameshift"]
            frame_list.append(frame)

            #species and genomes
            genome = prot.loc[p, "genome_id"]
            species = genomes.loc[genome,"species"]
            exec(species + "+= 1")
            genome_list.append(genome)

            #GO-terms
            if not pd.isnull(prot.loc[p, "GO_list"]):
                GOs = prot.loc[p,"GO_list"].split(";")
                for GO in GOs:
                    GO_rast.append(GO)

            #EC numbers
            if not pd.isnull(prot.loc[p, "EC_list"]):
                ECs=prot.loc[p,"EC_list"].split(";")
                for EC in ECs:
                    EC_rast.append(EC)
        
        # summary statistics
        GOdistr_rast = str(Counter(GO_rast))[8:-1]
        GO_rast = ';'.join(str(s) for s in set(GO_rast)) 
        ECdistr_rast = str(Counter(EC_rast))[8:-1]
        EC_rast = ';'.join(str(s) for s in set(EC_rast)) 
        prod_distr = str(Counter(prod_list))[8:-1]
        distribution = len(set(genome_list))
        with_frameshift = frame_list.count("yes")
        all_hypothetical = 'no'

        if all([re.search("hypothetic", m) for m in prod_list]):
            product = "hypothetical protein"
            all_hypothetical = "yes"
        else:
            f = re.match("(.+?): [0-9]+", prod_distr[1:-1]).group(1) # finds the most common product in the group
            if re.search("hypothetical", f):
                product="hypothetical protein"
            else:
                product=f[1:-1]    
        
        l = [group, OGsize, distribution, med_length, all_hypothetical, product, with_frameshift, prod_distr, trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum, avium, gallinacea, ibidis, suis, poikilothermis, sanzinia , serpentis, corallus, GOdistr_rast, GO_rast, ECdistr_rast, EC_rast]
        ogroups_withoutOutgroup.loc[i] = l
        i += 1

ogroups_withoutOutgroup.to_csv(outf, index = False)  

In [111]:
# 4B. genus Chlamydia without draft genomes

# column names in OG table (=> to add pfam)
col_list_nodraft = ["group_id", "group_size", "distribution", "trachomatis", "psittaci", "pneumoniae", "pecorum", "abortus", "caviae", "felis", "muridarum", "avium", "gallinacea", "ibidis", "suis", "poikilothermis", "sanzinia", "serpentis", "corallus"]
# output data.frame and file
ogroups_nodraft = pd.DataFrame(columns = col_list_nodraft)
outf = main_dir + '/data/ogroups2019/annotated/groups_without_outgroup_with_singletons_nodraft_annotated.txt'

i = 0
for group in og2prot_nodraft.keys():
    
    # list of proteins in the group
    proteins = og2prot_nodraft[group]
    
    # !! False if some genomes were excluded after OGs had been created !!
    if len(proteins) > 0: 
        
        # prepare
        genome_list = []
        trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum, avium, gallinacea, ibidis, suis, poikilothermis, sanzinia , serpentis, corallus   = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        product = ''
        distribution = 0
        
        # general statistics
        OGsize = len(proteins)

        # info by protein
        for p in proteins:

            #species and genomes
            genome = prot.loc[p, "genome_id"]
            species = genomes.loc[genome,"species"]
            exec(species + "+= 1")
            genome_list.append(genome)

        
        # summary statistics
        distribution = len(set(genome_list))
        with_frameshift = frame_list.count("yes")
        
        l = [group, OGsize, distribution, trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum, avium, gallinacea, ibidis, suis, poikilothermis, sanzinia , serpentis, corallus]

        ogroups_nodraft.loc[i] = l
        i += 1

ogroups_nodraft.to_csv(outf, index = False)  

In [115]:
# 4C. genus Chlamydia with outgroup

# column names in OG table (=> to add pfam)
col_list_withOutgroup = ["group_id", "group_size", "distribution", "trachomatis", "psittaci", "pneumoniae", "pecorum", "abortus", "caviae", "felis", "muridarum", "avium", "gallinacea", "ibidis", "suis", "poikilothermis", "sanzinia", "serpentis", "corallus", "Waddlia_chondrophila"]
# output data.frame and file
ogroups_withOutgroup = pd.DataFrame(columns = col_list_withOutgroup)
outf = main_dir + '/data/ogroups2019/annotated/groups_with_outgroup_with_singletons_annotated.txt'

i = 0
for group in og2prot_withOutgroup.keys():
    
    # list of proteins in the group
    proteins = og2prot_withOutgroup[group]
    
    # !! False if some genomes were excluded after OGs had been created !!
    if len(proteins) > 0: 
        
        # prepare
        genome_list = []
        trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum, avium, gallinacea, ibidis, suis, poikilothermis, sanzinia , serpentis, corallus, Waddlia_chondrophila   = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        product = ''
        distribution = 0
        
        # general statistics
        OGsize = len(proteins)

        # info by protein
        for p in proteins:

            #species and genomes
            genome = prot.loc[p, "genome_id"]
            species = genomes.loc[genome,"species"]
            exec(species + "+= 1")
            genome_list.append(genome)

        
        # summary statistics
        distribution = len(set(genome_list))
        with_frameshift = frame_list.count("yes")
        
        l = [group, OGsize, distribution, trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum, avium, gallinacea, ibidis, suis, poikilothermis, sanzinia , serpentis, corallus, Waddlia_chondrophila]

        ogroups_withOutgroup.loc[i] = l
        i += 1

ogroups_withOutgroup.to_csv(outf, index = False)  

In [81]:

        

        
#### TO BE ADDED #####        
#         #pfam info
#         if len(proteins)>1:
#             query="select * from pfam where protein_id in "+str(tuple(proteins))
#         else:
#             query="select * from pfam where protein_id='"+str(proteins[0])+"'"
#         df=pd.read_sql(query, connection)
#         pfams=df["accession"].tolist()
#         pfam_distr=str(Counter(pfams))[8:-1]
#         pfams=';'.join(str(s) for s in set(pfams)) 
#         #pfam to GO
#         GO_pfam=df["pfam2go"].tolist()
#         GO_pfam=[x.split("|") for x in GO_pfam]
#         GO_pfam=list(itertools.chain.from_iterable(GO_pfam))
#         GOdistr_pfam=str(Counter(GO_pfam))[8:-1]
#         GO_pfam=';'.join(str(s) for s in set(GO_pfam)) 



['S782', 1, 1, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 1}", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, '', '', '', '']
['S783', 1, 1, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 1}", 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', '']
['OG1449', 14, 14, 37.0, 'yes', 'hypothetical protein', 1, "{'FIG00493827: hypothetical protein': 14}", 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, '', '', '', '']
['OG1448', 14, 14, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 14}", 0, 0, 12, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, '', '', '', '']
['OG1339', 20, 20, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 20}", 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', '']
['OG1338', 20, 20, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 20}", 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', '']
['S784', 1, 1, 37.0, 'yes', 'hypothetical protein', 0, "{'hypot

['OG612', 234, 234, 37.0, 'no', 'DNA polymerase III, epsilon chain', 0, "{'DNA polymerase III, epsilon chain': 233, 'DNA polymerase III epsilon chain': 1}", 115, 25, 12, 10, 29, 1, 1, 3, 1, 2, 1, 30, 1, 1, 1, 1, '', '', '', '']
['OG539', 234, 234, 37.0, 'no', 'Peptide chain release factor 2; programmed frameshift-containing', 27, "{'Peptide chain release factor 2; programmed frameshift-containing': 233, 'Peptide chain release factor 2 @ programmed frameshift-containing': 1}", 115, 25, 12, 10, 29, 1, 1, 3, 1, 2, 1, 30, 1, 1, 1, 1, "{'GO:0006415': 234, 'GO:0003747': 234}", 'GO:0006415;GO:0003747', '', '']
['OG538', 234, 234, 37.0, 'no', 'Amino Group Acetyl Transferase', 0, "{'Amino Group Acetyl Transferase': 234}", 115, 25, 12, 10, 29, 1, 1, 3, 1, 2, 1, 30, 1, 1, 1, 1, '', '', '', '']
['S254', 1, 1, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 1}", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, '', '', '', '']
['S255', 1, 1, 37.0, 'yes', 'hypothetical protein', 0, "

['OG1151', 36, 36, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 36}", 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', '']
['OG1150', 36, 36, 37.0, 'yes', 'hypothetical protein', 16, "{'hypothetical protein': 36}", 0, 5, 0, 0, 29, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, '', '', '', '']
['OG1153', 36, 36, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 36}", 5, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 29, 0, 0, 0, 0, '', '', '', '']
['OG1152', 36, 36, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 36}", 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, '', '', '', '']
['OG1159', 34, 34, 37.0, 'yes', 'hypothetical protein', 0, "{'hypothetical protein': 34}", 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', '']
['OG1158', 34, 34, 37.0, 'no', 'phosphatidylcholine-hydrolyzing phospholipase D (PLD) family', 0, "{'phosphatidylcholine-hydrolyzing phospholipase D (PLD) family': 34}", 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:

# without outgroup, 128 genomes

for group in og2prot_withoutOutgroup.keys():
    proteins=og2prot_withoutOutgroup[group]
    if len(proteins)>0: # excluded proteins (if some genomes are to be excluded after OGs are created)
        OGsize=len(proteins)
        prod_list=[]
        frame_list=[]
        genome_list=[]
        GO_rast=[]
        EC_rast=[]
        trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum,avium,gallinacea, ibidis=[0,0,0,0,0,0,0,0,0,0,0]
        product=''
        distribution=0
        avg_length=prot[prot.index.isin(proteins)]["length"].mean()


        for p in proteins:

            #products
            prod=prot.loc[p,"product"]
            prod_list.append(prod)

            #frameshifts
            frame=prot.loc[p,"frameshift"]
            frame_list.append(frame)

            #species and genomes
            genome=prot.loc[p,"genome_id"]
            species=genomes.loc[genome,"species"]
            exec(species + "+= 1")
            genome_list.append(genome)


            #GO-terms
            if not pd.isnull(prot.loc[p,"RAST2GO"]):
                GOs=prot.loc[p,"RAST2GO"].split(";")
                for GO in GOs:
                    GO_rast.append(GO)


            #EC numbers
            if not pd.isnull(prot.loc[p,"RAST2EC"]):
                ECs=prot.loc[p,"RAST2EC"].split(";")
                for EC in ECs:
                    EC_rast.append(EC)
        
        #summary statistics
        GOdistr_rast=str(Counter(GO_rast))[8:-1]
        GO_rast=';'.join(str(s) for s in set(GO_rast)) 
        ECdistr_rast=str(Counter(EC_rast))[8:-1]
        EC_rast=';'.join(str(s) for s in set(EC_rast)) 
        prod_distr=str(Counter(prod_list))[8:-1]
        distribution=len(set(genome_list))
        with_frameshift=frame_list.count("yes")
        all_hypothetical='no'

        if all([re.search("hypothetic",m) for m in prod_list]):
            product="hypothetical protein"
            all_hypothetical="yes"
        else:
            f=re.match("(.+?): [0-9]+",prod_distr[1:-1]).group(1) # finds the most common product in the group
            if re.search("hypothetical",f):
                product="hypothetical protein"
            else:
                product=f[1:-1]    
        
        #pfam info
        if len(proteins)>1:
            query="select * from pfam where protein_id in "+str(tuple(proteins))
        else:
            query="select * from pfam where protein_id='"+str(proteins[0])+"'"
        df=pd.read_sql(query, connection)
        pfams=df["accession"].tolist()
        pfam_distr=str(Counter(pfams))[8:-1]
        pfams=';'.join(str(s) for s in set(pfams)) 
        #pfam to GO
        GO_pfam=df["pfam2go"].tolist()
        GO_pfam=[x.split("|") for x in GO_pfam]
        GO_pfam=list(itertools.chain.from_iterable(GO_pfam))
        GOdistr_pfam=str(Counter(GO_pfam))[8:-1]
        GO_pfam=';'.join(str(s) for s in set(GO_pfam)) 

        #add to database
        out_list=[group, OGsize, distribution,avg_length, all_hypothetical,product,with_frameshift,prod_distr,trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum,avium,gallinacea, ibidis, pfam_distr,pfams,GOdistr_pfam,GO_pfam, GOdistr_rast,GO_rast, ECdistr_rast,EC_rast]
        cursor.execute(sql_og_withoutOutgroup,(out_list))
        #out_list=[str(x) for x in out_list]
        #out_list="\t".join(out_list)+"\n"


# with outgroup, 129 genomes

for group in og2prot_withOutgroup.keys():
    proteins=og2prot_withOutgroup[group]
    if len(proteins)>0: # excluded proteins (if some genomes are to be excluded after OGs are created)
        OGsize=len(proteins)
        prod_list=[]
        frame_list=[]
        genome_list=[]
        GO_rast=[]
        EC_rast=[]
        trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum, avium, gallinacea, ibidis, Waddlia_chondrophila = [0,0,0,0,0,0,0,0,0,0,0,0]
        product=''
        distribution=0
        avg_length=prot[prot.index.isin(proteins)]["length"].mean()


        for p in proteins:

            #products
            prod=prot.loc[p,"product"]
            prod_list.append(prod)

            #frameshifts
            frame=prot.loc[p,"frameshift"]
            frame_list.append(frame)

            #species and genomes
            genome=prot.loc[p,"genome_id"]
            species=genomes.loc[genome,"species"]
            exec(species + "+= 1")
            genome_list.append(genome)


            #GO-terms
            if not pd.isnull(prot.loc[p,"RAST2GO"]):
                GOs=prot.loc[p,"RAST2GO"].split(";")
                for GO in GOs:
                    GO_rast.append(GO)


            #EC numbers
            if not pd.isnull(prot.loc[p,"RAST2EC"]):
                ECs=prot.loc[p,"RAST2EC"].split(";")
                for EC in ECs:
                    EC_rast.append(EC)
        
        #summary statistics
        GOdistr_rast=str(Counter(GO_rast))[8:-1]
        GO_rast=';'.join(str(s) for s in set(GO_rast)) 
        ECdistr_rast=str(Counter(EC_rast))[8:-1]
        EC_rast=';'.join(str(s) for s in set(EC_rast)) 
        prod_distr=str(Counter(prod_list))[8:-1]
        distribution=len(set(genome_list))
        with_frameshift=frame_list.count("yes")
        all_hypothetical='no'

        if all([re.search("hypothetic",m) for m in prod_list]):
            product="hypothetical protein"
            all_hypothetical="yes"
        else:
            f=re.match("(.+?): [0-9]+",prod_distr[1:-1]).group(1) # finds the most common product in the group
            if re.search("hypothetical",f):
                product="hypothetical protein"
            else:
                product=f[1:-1]    
        
        #pfam info
        if len(proteins)>1:
            query="select * from pfam where protein_id in "+str(tuple(proteins))
        else:
            query="select * from pfam where protein_id='"+str(proteins[0])+"'"
        df=pd.read_sql(query, connection)
        pfams=df["accession"].tolist()
        pfam_distr=str(Counter(pfams))[8:-1]
        pfams=';'.join(str(s) for s in set(pfams)) 
        #pfam to GO
        GO_pfam=df["pfam2go"].tolist()
        GO_pfam=[x.split("|") for x in GO_pfam]
        GO_pfam=list(itertools.chain.from_iterable(GO_pfam))
        GOdistr_pfam=str(Counter(GO_pfam))[8:-1]
        GO_pfam=';'.join(str(s) for s in set(GO_pfam)) 

        #add to database
        out_list=[group, OGsize, distribution,avg_length, all_hypothetical,product,with_frameshift,prod_distr,trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum,avium,gallinacea, ibidis,Waddlia_chondrophila, pfam_distr,pfams,GOdistr_pfam,GO_pfam, GOdistr_rast,GO_rast, ECdistr_rast,EC_rast]
        cursor.execute(sql_og_withOutgroup,(out_list))
        #out_list=[str(x) for x in out_list]
        #out_list="\t".join(out_list)+"\n"


# without outgroup, no drafts



for group in og2prot_nodraft.keys():
    proteins=og2prot_nodraft[group]
    if len(proteins)>0: # excluded proteins (if some genomes are to be excluded after OGs are created)
        OGsize=len(proteins)
        prod_list=[]
        frame_list=[]
        genome_list=[]
        GO_rast=[]
        EC_rast=[]
        trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum,avium,gallinacea, ibidis, Waddlia_chondrophila = [0,0,0,0,0,0,0,0,0,0,0,0]
        product=''
        distribution=0
        avg_length=prot[prot.index.isin(proteins)]["length"].mean()
        for p in proteins:
            #products
            prod=prot.loc[p,"product"]
            prod_list.append(prod)
            #frameshifts
            frame=prot.loc[p,"frameshift"]
            frame_list.append(frame)
            #species and genomes
            genome=prot.loc[p,"genome_id"]
            species=genomes.loc[genome,"species"]
            exec(species + "+= 1")
            genome_list.append(genome)
            #GO-terms
            if not pd.isnull(prot.loc[p,"RAST2GO"]):
                GOs=prot.loc[p,"RAST2GO"].split(";")
                for GO in GOs:
                    GO_rast.append(GO)
            #EC numbers
            if not pd.isnull(prot.loc[p,"RAST2EC"]):
                ECs=prot.loc[p,"RAST2EC"].split(";")
                for EC in ECs:
                    EC_rast.append(EC)
        #summary statistics
        GOdistr_rast=str(Counter(GO_rast))[8:-1]
        GO_rast=';'.join(str(s) for s in set(GO_rast)) 
        ECdistr_rast=str(Counter(EC_rast))[8:-1]
        EC_rast=';'.join(str(s) for s in set(EC_rast)) 
        prod_distr=str(Counter(prod_list))[8:-1]
        distribution=len(set(genome_list))
        with_frameshift=frame_list.count("yes")
        all_hypothetical='no'
        if all([re.search("hypothetic",m) for m in prod_list]):
            product="hypothetical protein"
            all_hypothetical="yes"
        else:
            f=re.match("(.+?): [0-9]+",prod_distr[1:-1]).group(1) # finds the most common product in the group
            if re.search("hypothetical",f):
                product="hypothetical protein"
            else:
                product=f[1:-1]    
        #pfam info
        if len(proteins)>1:
            query="select * from pfam where protein_id in "+str(tuple(proteins))
        else:
            query="select * from pfam where protein_id='"+str(proteins[0])+"'"
        df=pd.read_sql(query, connection)
        pfams=df["accession"].tolist()
        pfam_distr=str(Counter(pfams))[8:-1]
        pfams=';'.join(str(s) for s in set(pfams)) 
        #pfam to GO
        GO_pfam=df["pfam2go"].tolist()
        GO_pfam=[x.split("|") for x in GO_pfam]
        GO_pfam=list(itertools.chain.from_iterable(GO_pfam))
        GOdistr_pfam=str(Counter(GO_pfam))[8:-1]
        GO_pfam=';'.join(str(s) for s in set(GO_pfam)) 
        #add to database
        out_list=[group, OGsize, distribution,avg_length, all_hypothetical,product,with_frameshift,prod_distr,trachomatis, psittaci, pneumoniae, pecorum, abortus, caviae, felis, muridarum,avium,gallinacea, ibidis,pfam_distr,pfams,GOdistr_pfam,GO_pfam, GOdistr_rast,GO_rast, ECdistr_rast,EC_rast]
        cursor.execute(sql_og_nodraft,(out_list))




In [ ]:

# ######### Pfams ##################################################

# print ("parsing pfams info\n\n")

# pfam=pd.read_csv(main_dir+"info/InterProScan_results/Chlamydia_interproscan_w_outgroup_pfam.tsv",sep="\t", names=["protein_id", "Sequence_MD5_digest", "length", "analysis", "accession", "description", "start", "stop", "score", "status", "date", "ipr_annotation", "ipr_description", "GO"],index_col=False)
# ids=set([x.split("|")[1] for x in pfam["protein_id"].tolist() if x.split("|")[0] in genome_ids_withOutgroup])
# pfam_dict=dict((zip(ids,[1]*len(ids))))

# for i in range (pfam.shape[0]):
#     p=pfam.loc[i,"protein_id"].split("|")[1]
#     try:
#         pfam["id_short"]=p
#         num=pfam_dict[p]
#         pfam_dict[p]+=1
#         pfam_id=p+"."+str(num)
#         acc=pfam.loc[i,"accession"]
#         desc=pfam.loc[i,"description"]
#         start=pfam_descr=pfam.loc[i,"start"]
#         end=pfam.loc[i,"stop"]
#         pfam2go=pfam.loc[i,"GO"]
#         if pd.isnull(pfam2go):
#             pfam2go=""
#         cursor.execute(sql_pfam,(pfam_id,p,acc,desc,start,end,pfam2go))
#     except:
#         pass

#################### Orthologous groups ################################

